In [1]:
from scipy.optimize import root
import numpy as np
import pickle

In [2]:
with open('mf1.pkl', 'rb') as f:
    ss_params,model_params,ss_shock = pickle.load(f)


In [3]:
def assign_params(ss_params):
    a_0 = ss_params['a_0']
    a_1 = ss_params['a_1']
    a_2 = ss_params['a_2']
    alf = ss_params['alf']
    delt = ss_params['delt']
    om_0 = ss_params['om_0']
    om_1 = ss_params['om_1']
    om_2 = ss_params['om_2']
    eps = ss_params['eps']
    tht = ss_params['tht']
    bet = ss_params['bet']
    psi_k = ss_params['psi_k']
    psi_n = ss_params['psi_n']
    return eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_k,psi_n,a_0,a_1,a_2


In [4]:
def equations(vars, params):
	n_0,n_1,n_2,s_0,s_1,s_2,k,k_0,k_1,k_2,c_0,c_1,c_2,i = vars
	eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_k,psi_n,a_0,a_1,a_2 = assign_params(params)
 
	
	ph=(om_0*c_0**((eps-1)/eps)+om_1*c_1**((eps-1)/eps)+om_2*c_2**((eps-1)/eps)	)**((1-tht*eps)/(eps-1))

	v_0=om_0*c_0**(-1/eps)*ph
	v_1=om_1*c_1**(-1/eps)*ph
	v_2=om_2*c_2**(-1/eps)*ph

	q_0=(1-psi_n/2*(n_0/n_0-1)**2)
	q_1=(1-psi_n/2*(n_1/n_1-1)**2)
	q_2=(1-psi_n/2*(n_2/n_2-1)**2)


	fn_0=np.exp(a_0)*(1-alf)*(s_0/n_0/q_0)**alf*k**alf
	fn_1=np.exp(a_1)*(1-alf)*(s_1/n_1/q_1)**alf*k**alf
	fn_2=np.exp(a_2)*(1-alf)*(s_2/n_2/q_2)**alf*k**alf
 
	fs_0=np.exp(a_0)*alf*(n_0/s_0*q_0)**(1-alf)*k**alf
	fs_1=np.exp(a_1)*alf*(n_1/s_1*q_1)**(1-alf)*k**alf
	fs_2=np.exp(a_2)*alf*(n_2/s_2*q_2)**(1-alf)*k**alf
  
	x_10=k_0*q_0**(1-alf)-n_0*psi_n/q_0**alf/n_0*(n_0/n_0-1)
	x_11=k_1*q_1**(1-alf)-n_1*psi_n/q_1**alf/n_1*(n_1/n_1-1)
	x_12=k_2*q_2**(1-alf)-n_2*psi_n/q_2**alf/n_2*(n_2/n_2-1)

	x_20=psi_n*(n_0/n_0-1)*(n_0/n_0)**2
	x_21=psi_n*(n_1/n_1-1)*(n_1/n_1)**2
	x_22=psi_n*(n_2/n_2-1)*(n_2/n_2)**2
 
	z_1=1-psi_k*(i/k-delt)
	z_2=1-delt-psi_k/2*(i/k-delt)**2+psi_k*i/k*(i/k-delt)
 
	eq1=n_0+n_1+n_2-1
	eq2=s_0+s_1+s_2-1
	eq3=k_0-s_0/n_0*k
	eq4=k_1-s_1/n_1*k
	eq5=k_2-s_2/n_2*k
	eq6=v_1*x_11*fn_1+v_1*x_21*fn_1-(v_0*x_10*fn_0+v_0*x_20*fn_0)
	eq7=v_2*x_12*fn_2+v_2*x_22*fn_2-(v_0*x_10*fn_0+v_0*x_20*fn_0)
	eq8=v_1*fs_1-v_0*fs_0
	eq9=v_2*fs_2-v_0*fs_0
	eq10=bet*(fs_0/k+z_2/z_1)-(v_0/v_0/z_1)
	eq11=(1-delt)*k+i-psi_k/2*(i/k-delt)**2*k-k
	eq12=c_0+i-np.exp(a_0)*n_0**(1-alf)*s_0**alf*k**alf
	eq13=c_1-np.exp(a_1)*n_1**(1-alf)*s_1**alf*k**alf
	eq14=c_2-np.exp(a_2)*n_2**(1-alf)*s_2**alf*k**alf
	
	return [eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9,eq10,eq11,eq12,eq13,eq14]


In [5]:
initial_guess = [.3,.3,.3,.3,.3,.3,3,.3,.3,.3,.3,.3,.3,0]
def solve_equations(params):
    
    sol = root(equations, initial_guess, args=(ss_params,))
    
    # Extract the solution vector
    n_0,n_1,n_2,s_0,s_1,s_2,k,k_0,k_1,k_2,c_0,c_1,c_2,i = sol.x
    eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_k,psi_n,a_0,a_1,a_2 = assign_params(params)
    
    y_0=c_0+i
    y=y_0+om_1/om_0*(c_1/c_0)**(-1/eps)*c_1+om_2/om_0*(c_2/c_0)**(-1/eps)*c_2
    r=np.exp(a_0)*alf*k**(alf-1)*(s_0/n_0)**alf+1-delt
    tot=0
    sc=0
    win=0
    p_0=1
    p_1=1
    p_2=1
    ex_0=1
    ex_1=1
    ex_2=1
    exb_0=0
    exb_1=0
    exb_2=0
    a_0=0
    a_1=0
    a_2=0
    ss_dict = {
    'n_0': n_0,
    'n_1': n_1,
    'n_2': n_2,
    's_0': s_0,
    's_1': s_1,
    's_2': s_2,    
    'k': k,
    'k_0': k_0,
    'k_1': k_1,
    'k_2': k_2,
    'c_0': c_0,
    'c_1': c_1,
    'c_2': c_2,
    'i': i,
    'y_0':y_0,
    'y': y,
    'r': r,
    'tot':tot,
    'sc':sc,
    'win':win,
    'p_0':p_0,
    'p_1':p_1,
    'p_2':p_2,
    'ex_0':ex_0,
    'ex_1':ex_1,
    'ex_2':ex_2,
    'exb_0':exb_0,
    'exb_1':exb_1,
    'exb_2':exb_2,
     
    
    
        
    'a_0': a_0,
    'a_1': a_1,
    'a_2': a_2
    }
    return sol, ss_dict
    

# Call the wrapper function
sol, ss_dict=solve_equations(ss_params)
print(sol)
ss_dict


 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00 ...  2.634e-11 -4.198e-12]
       x: [ 2.663e-01  3.464e-01 ...  5.055e-01  1.215e-01]
    nfev: 61
    fjac: [[-1.159e-01  5.538e-11 ... -7.075e-04 -1.825e-03]
           [-1.610e-01 -1.959e-10 ...  1.344e-01 -4.294e-03]
           ...
           [-3.383e-01 -1.639e-02 ... -3.084e-01 -3.217e-01]
           [-6.421e-02  9.228e-02 ...  2.426e-02  2.031e-02]]
       r: [-8.630e+00  2.177e-01 ...  2.101e-01 -7.394e-01]
     qtf: [ 5.891e-09  1.761e-08 ... -7.917e-10  1.706e-10]


C:\Users\blankenw\AppData\Local\Temp\ipykernel_8948\781936935.py:17: RuntimeWarning: invalid value encountered in scalar power
  fn_0=np.exp(a_0)*(1-alf)*(s_0/n_0/q_0)**alf*k**alf
C:\Users\blankenw\AppData\Local\Temp\ipykernel_8948\781936935.py:21: RuntimeWarning: invalid value encountered in scalar power
  fs_0=np.exp(a_0)*alf*(n_0/s_0*q_0)**(1-alf)*k**alf
C:\Users\blankenw\AppData\Local\Temp\ipykernel_8948\781936935.py:47: RuntimeWarning: invalid value encountered in scalar power
  eq12=c_0+i-np.exp(a_0)*n_0**(1-alf)*s_0**alf*k**alf


{'n_0': 0.2663052775252695,
 'n_1': 0.3464036584238632,
 'n_2': 0.38729106405086733,
 's_0': 0.26630527753323957,
 's_1': 0.3464036584845492,
 's_2': 0.38729106398221125,
 'k': 2.4305705489991745,
 'k_0': 2.4305705490651044,
 'k_1': 2.4305705488325544,
 'k_2': 2.4305705490197496,
 'c_0': 0.22608145725018503,
 'c_1': 0.4521629144531625,
 'c_2': 0.5055335068616944,
 'i': 0.12152852744995817,
 'y_0': 0.3476099847001432,
 'y': 1.3053064061132793,
 'r': 1.1111111111196683,
 'tot': 0,
 'sc': 0,
 'win': 0,
 'p_0': 1,
 'p_1': 1,
 'p_2': 1,
 'ex_0': 1,
 'ex_1': 1,
 'ex_2': 1,
 'exb_0': 0,
 'exb_1': 0,
 'exb_2': 0,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [6]:
# Update the dictionary values
for key, value in ss_dict.items():
    if value != 0:
        ss_dict[key] = np.log(value)

# Print the updated dictionary
print(ss_dict)

{'n_0': -1.3231119683891552, 'n_1': -1.0601505410798138, 'n_2': -0.9485787651498407, 's_0': -1.323111968359227, 's_1': -1.0601505409046252, 's_2': -0.9485787653271133, 'k': 0.8881260236195211, 'k_0': 0.8881260236466464, 'k_1': 0.8881260235509693, 'k_2': 0.8881260236279862, 'c_0': -1.4868599143386987, 'c_1': -0.7937127338831573, 'c_2': -0.6821409581254703, 'i': -2.1076062499344745, 'y_0': -1.0566741613194897, 'y': 0.2664378071714054, 'r': 0.10536051566552779, 'tot': 0, 'sc': 0, 'win': 0, 'p_0': 0.0, 'p_1': 0.0, 'p_2': 0.0, 'ex_0': 0.0, 'ex_1': 0.0, 'ex_2': 0.0, 'exb_0': 0, 'exb_1': 0, 'exb_2': 0, 'a_0': 0, 'a_1': 0, 'a_2': 0}


In [7]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('var \n')
        for key, value in ss_dict.items():
            f.write(f'{key}\n')
        f.write(f';\n\n\n')
        f.write('varexo \n')
        for key, value in ss_shock.items():
            f.write(f'{key}_shock\n')
        f.write(f';\n\n\n')
# Call the function
write_dict_to_file(ss_dict, 't_vars.m')

In [8]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('initval;\n')
        for key, value in ss_dict.items():
            f.write(f'{key}={value};\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_ss.m')

In [9]:
ss_dict

{'n_0': -1.3231119683891552,
 'n_1': -1.0601505410798138,
 'n_2': -0.9485787651498407,
 's_0': -1.323111968359227,
 's_1': -1.0601505409046252,
 's_2': -0.9485787653271133,
 'k': 0.8881260236195211,
 'k_0': 0.8881260236466464,
 'k_1': 0.8881260235509693,
 'k_2': 0.8881260236279862,
 'c_0': -1.4868599143386987,
 'c_1': -0.7937127338831573,
 'c_2': -0.6821409581254703,
 'i': -2.1076062499344745,
 'y_0': -1.0566741613194897,
 'y': 0.2664378071714054,
 'r': 0.10536051566552779,
 'tot': 0,
 'sc': 0,
 'win': 0,
 'p_0': 0.0,
 'p_1': 0.0,
 'p_2': 0.0,
 'ex_0': 0.0,
 'ex_1': 0.0,
 'ex_2': 0.0,
 'exb_0': 0,
 'exb_1': 0,
 'exb_2': 0,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [10]:
def write_dict_to_file(params, filename):
    
    with open(filename, 'w') as f:
        f.write('parameters ')
        f.write(','.join(f'{key}' for key in params))
        f.write(';\n\n')
        for key, value in params.items():
            f.write(f'{key}={value};\n')

# Call the function
write_dict_to_file(model_params, 't_parameters.m')

In [11]:
def write_dict_to_file(ss_dict, filename):
    #variance_list = list(shock_params.values())[:3]
    with open(filename, 'w') as f:
        f.write('shocks;\n')
        i=0
        for key, value in ss_shock.items():
            f.write(f'var {key}_shock=var_{key};\n')
            i+=1
        f.write(f'\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_shocks.m')

In [12]:
with open('mf2.pkl', 'wb') as f:
    pickle.dump((ss_dict), f)